In [1]:
!pip install peft
!pip install evaluate
!pip install seqeval
from transformers import (AutoModelForSequenceClassification, BertForTokenClassification, AutoTokenizer,
                          TrainingArguments, Trainer, DataCollatorForTokenClassification,
                         TrainingArguments,Trainer)
import pandas as pd
import evaluate
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from peft import LoraModel, LoraConfig, get_peft_model
seqeval = evaluate.load("seqeval")
BERT = BertForTokenClassification.from_pretrained('google-bert/bert-base-cased',num_labels = 2)
Tokenizer =  AutoTokenizer.from_pretrained('google-bert/bert-base-cased')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=13badebf1a5eba0ed26abd395d677fb95070c6db2ca62b72e7c301d98ad4d946
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


2024-07-11 11:25:55.990370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 11:25:55.990520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 11:25:56.167525: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet, words
from nltk.tokenize import regexp_tokenize
pattern = r"\b\w+(?:'\w+)?\b"
import random
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

def label_generator(txt):
    c = word_tokenize(txt[0])
    i = word_tokenize(txt[1])
    label = []
    for w1,w2 in zip(c,i):
        if w1!=w2:
            label.append(1)
        else:
            label.append(0)

    return label

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [102]:
import random
contractions = {
    "I'll": ["isle", "aisle"],
    "we'll": ["weal", "wheel"],
    "he'll": ["heal", "heel"],
    "you'll": ["yule"],
    "aren't": ["aunt"],
    "he'd": ["heed"],
    "it's": ["its"],
    "they're": ["their", "there"],
    "who's": ["whose"],
    "we'd" : ["weed"],
    "you're": ["yore","yaw","your"],
    "cannot": ["a"],
    "daydreams'":["daydreamed"]
}


# Replace contractions in Data["Errors"]
for original, replacements in contractions.items():
    Data["Errors"] = Data["Errors"].str.replace(original, random.choice(replacements))

In [101]:
Data = pd.read_csv("/kaggle/input/spellcheckdataset/Final Training dataset_1.csv")

In [103]:
#Data = pd.read_csv("/kaggle/input/spell-check-data/Final Training dataset.csv")
from tqdm import tqdm

tqdm.pandas(desc="Processing rows")

Data["Label"] = Data[['Final','Errors']].progress_apply(label_generator,axis=1)

Processing rows:   0%|          | 0/65735 [00:00<?, ?it/s]/tmp/ipykernel_33/969590983.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c = word_tokenize(txt[0])
/tmp/ipykernel_33/969590983.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  i = word_tokenize(txt[1])
Processing rows: 100%|██████████| 65735/65735 [02:14<00:00, 489.06it/s]


In [12]:
Data["Label"].loc[1313]

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [104]:
# remove if the mask is more than 50 %, Useful information can not be obtained from them
index = []
for i,items in enumerate(Data["Label"].values):
    if(sum(items)/len(items)>0.5):
        index.append(i)
        
Data.drop(index,inplace=True)
Data.reset_index(drop=True,inplace=True)

In [105]:
# Calculate consecutive ones maxlen subarray. If more than 3 trim off. because context will be lost completely
consec_ones = []
for i,items in enumerate(Data["Label"].values):
    max_len = 0
    counter = 0
    for j in items:
        if j==1:
            counter+=1
        else:
            max_len = max(max_len,counter)
            counter=0
    max_len = max(max_len,counter)
    consec_ones.append(max_len)

In [106]:
import numpy as np
arr = np.array(consec_ones)
indices = np.where(arr > 3)[0]
print(len(indices))
#print(indices)
Data.drop(indices,inplace = True)
Data.reset_index(inplace = True,drop=True)

4507


In [107]:
Train_Data, Test_Data = train_test_split(Data,test_size=0.2)
Train_Data.reset_index(inplace=True,drop=True)
Test_Data.reset_index(inplace=True,drop=True)
Train_data = Train_Data[:40000]
Validation_data = Train_Data[40000:]

Text_train = Train_data["Errors"].values.tolist()
Label_train = Train_data["Label"].values.tolist()

Text_val = Validation_data["Errors"].values.tolist()
Label_val = Validation_data["Label"].values.tolist()

Text_test = Test_Data["Errors"].values.tolist()
Label_test = Test_Data["Label"].values.tolist()

data_dict = {"Text":Text_train,"Label":Label_train}
ds_train = Dataset.from_dict(data_dict)

data_dict = {"Text":Text_val,"Label":Label_val}
ds_val = Dataset.from_dict(data_dict)

data_dict = {"Text":Text_test,"Label":Label_test}
ds_test = Dataset.from_dict(data_dict)

dataset = DatasetDict({
    'train': ds_train,
    'test': ds_test,
    'validation': ds_val
})

In [108]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'Label'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['Text', 'Label'],
        num_rows: 12231
    })
    validation: Dataset({
        features: ['Text', 'Label'],
        num_rows: 8920
    })
})

In [3]:
#count = 0
def tokenize_and_align_labels(example):
    #global count
    
    tokenized_text = word_tokenize(example["Text"])
    tokenized_inputs = Tokenizer(tokenized_text, truncation=True ,is_split_into_words=True)

    label = example["Label"]

    # Map tokens to their respective word indices
    word_ids = tokenized_inputs.word_ids()
    
    previous_word_idx = None
    label_ids = []

    # Align labels with tokenized inputs
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(label[word_idx])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = label_ids

    return tokenized_inputs

In [4]:
new = dataset.map(tokenize_and_align_labels)

NameError: name 'dataset' is not defined

In [111]:
new

DatasetDict({
    train: Dataset({
        features: ['Text', 'Label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['Text', 'Label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 12231
    })
    validation: Dataset({
        features: ['Text', 'Label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8920
    })
})

In [5]:
label_map = {0: 'CORRECT', 1: 'INCORRECT'}

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_map[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_map[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [6]:
data_collator = DataCollatorForTokenClassification(tokenizer=Tokenizer)

# Freezing the layers

In [114]:
BERT = BertForTokenClassification.from_pretrained('google-bert/bert-base-cased',num_labels = 2)
# Useless when using PEFT as it automatically freezes these weights
for params in BERT.bert.parameters():
    params.requires_grad = False
    
for params in BERT.bert.encoder.layer[0].parameters():
    params.requires_grad = True
    
"""for params in BERT.bert.encoder.layer[5].parameters():
    params.requires_grad = True"""
    
for params in BERT.bert.encoder.layer[11].parameters():
    params.requires_grad = True
    
for name, param in BERT.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert.embeddings.word_embeddings.weight: requires_grad = False
bert.embeddings.position_embeddings.weight: requires_grad = False
bert.embeddings.token_type_embeddings.weight: requires_grad = False
bert.embeddings.LayerNorm.weight: requires_grad = False
bert.embeddings.LayerNorm.bias: requires_grad = False
bert.encoder.layer.0.attention.self.query.weight: requires_grad = True
bert.encoder.layer.0.attention.self.query.bias: requires_grad = True
bert.encoder.layer.0.attention.self.key.weight: requires_grad = True
bert.encoder.layer.0.attention.self.key.bias: requires_grad = True
bert.encoder.layer.0.attention.self.value.weight: requires_grad = True
bert.encoder.layer.0.attention.self.value.bias: requires_grad = True
bert.encoder.layer.0.attention.output.dense.weight: requires_grad = True
bert.encoder.layer.0.attention.output.dense.bias: requires_grad = True
bert.encoder.layer.0.attention.output.LayerNorm.weight: requires_grad = True
bert.encoder.layer.0.attention.output.LayerNorm.bias: req

In [13]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )


In [116]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/",         
    num_train_epochs=10,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,    
    warmup_steps=50,                
    weight_decay=0.01,               
    logging_dir="/kaggle/working/",            
    logging_steps=10,                
    save_steps=500,                 
    save_total_limit=3,              
    eval_steps=500,                  
    load_best_model_at_end=True, 
    evaluation_strategy = "steps",
    report_to=[]
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [120]:
trainer = Trainer(
    model=BERT,                     
    args=training_args,               
    train_dataset=new["train"],
    eval_dataset=new["validation"],
    tokenizer=Tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )

In [121]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [25]:
trainer.evaluate(eval_dataset=new["validation"])

{'eval_loss': 0.03143584355711937,
 'eval_precision': 0.9492524153689491,
 'eval_recall': 0.9467598900146151,
 'eval_f1': 0.9480045143367397,
 'eval_accuracy': 0.990143401139718,
 'eval_runtime': 68.4311,
 'eval_samples_per_second': 130.35,
 'eval_steps_per_second': 8.154,
 'epoch': 10.0}

In [26]:
trainer.evaluate(eval_dataset=new["train"])

{'eval_loss': 0.014110065996646881,
 'eval_precision': 0.974600659594742,
 'eval_recall': 0.9752701882043235,
 'eval_f1': 0.9749353089512635,
 'eval_accuracy': 0.9952984186447256,
 'eval_runtime': 310.452,
 'eval_samples_per_second': 128.844,
 'eval_steps_per_second': 8.053,
 'epoch': 10.0}

In [27]:
trainer.evaluate(eval_dataset=new["test"])

{'eval_loss': 0.030832529067993164,
 'eval_precision': 0.9502667983744018,
 'eval_recall': 0.9484699918513616,
 'eval_f1': 0.9493675449387823,
 'eval_accuracy': 0.9904092466031396,
 'eval_runtime': 94.2764,
 'eval_samples_per_second': 129.736,
 'eval_steps_per_second': 8.114,
 'epoch': 10.0}

In [93]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query","value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier","query"],
)

In [18]:
def get_incorrect_word(model , sentence):
    tokenized_text = word_tokenize(sentence)
    #print(tokenized_text)
    tokenized_inputs = Tokenizer(tokenized_text, truncation=True ,is_split_into_words=True)
    
    input_ids = torch.tensor(tokenized_inputs['input_ids']).unsqueeze(0)
    
    prediction = model(input_ids).logits.argmax(2)[0]
    
    #print(prediction)
    
    word_ids = tokenized_inputs.word_ids()
    
    previous_word_idx = None
    
    true_predictions = []
    
    for i,word_idx in enumerate(word_ids):
        #print(word_idx)
        if word_idx != None and word_idx != previous_word_idx:
            true_predictions.append(prediction[i].item())
        previous_word_idx = word_idx
        
    true_predictions = torch.tensor(true_predictions)
    
    indices = torch.nonzero(true_predictions == 1, as_tuple=False).squeeze()

    if indices.numel() == 0:
        return "No incorrect words found"
        
    if indices.dim() == 0:  
        indices = indices.unsqueeze(0)
    
    incorrect_words = [tokenized_text[i.item()] for i in indices]
        
    return incorrect_words

In [29]:
import torch
Reloaded_BERTModel = BertForTokenClassification.from_pretrained('/kaggle/working/checkpoint-12000')

In [30]:
Reloaded_BERTModel.device

device(type='cpu')

In [113]:
get_incorrect_word(Reloaded_BERTModel , "The car was collided by a truck")

'No incorrect words found'

In [114]:
get_incorrect_word(Reloaded_BERTModel , "The car has collided by a truck")

['has']

In [110]:
# Still giving some wrong outputs, so we need further finetuning
get_incorrect_word(Reloaded_BERTModel , "The car collided a truck")

['collided']

In [140]:
get_incorrect_word(Reloaded_BERTModel , "The cat ate two mice")

'No incorrect words found'

## WikiPedia-data for finetuning

In [7]:
from tqdm import tqdm
Data_wiki = pd.read_csv("/kaggle/input/wikipedia-datase-for-spell-correction/wiki_dataset_with_errors.csv")

tqdm.pandas(desc="Processing rows")

Data_wiki["Label"] = Data_wiki[['Final','Errors']].progress_apply(label_generator,axis=1)

Processing rows:   0%|          | 0/30000 [00:00<?, ?it/s]/tmp/ipykernel_34/969590983.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c = word_tokenize(txt[0])
/tmp/ipykernel_34/969590983.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  i = word_tokenize(txt[1])
Processing rows: 100%|██████████| 30000/30000 [01:55<00:00, 260.59it/s]


In [8]:
consec_ones = []
for i,items in enumerate(Data_wiki["Label"].values):
    max_len = 0
    counter = 0
    for j in items:
        if j==1:
            counter+=1
        else:
            max_len = max(max_len,counter)
            counter=0
    max_len = max(max_len,counter)
    consec_ones.append(max_len)


import numpy as np
arr = np.array(consec_ones)
indices = np.where(arr > 3)[0]
print(len(indices))
#print(indices)
"""Data.drop(indices,inplace = True)
Data.reset_index(inplace = True,drop=True)"""

1514


'Data.drop(indices,inplace = True)\nData.reset_index(inplace = True,drop=True)'

In [9]:
Data_wiki.drop(indices,inplace = True)
Data_wiki.reset_index(inplace = True,drop=True)

In [10]:
Data_wiki.drop(12347,inplace = True)
Data_wiki.reset_index(inplace=True,drop=True)

Train_data = Data_wiki[:25000]
Validation_data = Data_wiki[25000:]

Text_train = Train_data["Errors"].values.tolist()
Label_train = Train_data["Label"].values.tolist()


data_dict = {"Text":Text_train,"Label":Label_train}
ds_train = Dataset.from_dict(data_dict)

Text_val = Validation_data["Errors"].values.tolist()
Label_val = Validation_data["Label"].values.tolist()

data_dict = {"Text":Text_val,"Label":Label_val}
ds_val = Dataset.from_dict(data_dict)

dataset = DatasetDict({
    'train': ds_train,
    'validation': ds_val
})

In [11]:
new = dataset.map(tokenize_and_align_labels)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3485 [00:00<?, ? examples/s]

In [15]:
new["train"] = new["train"].remove_columns(["Text", "Label"])
new['validation'] = new['validation'].remove_columns(["Text", "Label"])

In [14]:
#!pip install peft
from peft import LoraModel, LoraConfig, get_peft_model, PeftModel
from peft import PeftConfig
import copy
config = LoraConfig(
    task_type="TOKEN_CLS",
    r=8,
    lora_alpha=32,
    target_modules=["query","value"],
    lora_dropout=0.01,
    modules_to_save = ["classifier"]
)
BERT = BertForTokenClassification.from_pretrained("/kaggle/input/bert-trained-spellcheck/Bert for spell check")
print(print_trainable_parameters(BERT))
lora_model = get_peft_model(copy.deepcopy(BERT), config, adapter_name = "LORA")
print_trainable_parameters(lora_model)

trainable params: 107721218 || all params: 107721218 || trainable%: 100.00
None
trainable params: 296450 || all params: 108017668 || trainable%: 0.27


In [15]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/",         
    num_train_epochs=10,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,    
    warmup_steps=50,                
    weight_decay=0.01,               
    logging_dir="/kaggle/working/",            
    logging_steps=10,                
    save_steps=500,                 
    save_total_limit=3,              
    eval_steps=500,                  
    load_best_model_at_end=True, 
    evaluation_strategy = "steps",
    report_to=[],
)

#BERT = BertForTokenClassification.from_pretrained("/kaggle/input/bert-trained-spellcheck/Bert for spell check")

trainer = Trainer(
    model=lora_model,                     
    args=training_args,               
    train_dataset=new["train"],
    eval_dataset = new["validation"],
    tokenizer=Tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.032800,0.026721,0.913661,0.891519,0.902454,0.990250
1000,0.027900,0.024128,0.921790,0.903936,0.912776,0.991305
1500,0.025100,0.022724,0.926900,0.910067,0.918406,0.991887
2000,0.022000,0.021825,0.928102,0.915869,0.921945,0.992242
2500,0.024100,0.021251,0.931479,0.915743,0.923544,0.992417
3000,0.019700,0.020818,0.933566,0.917397,0.925411,0.992608
3500,0.022500,0.020437,0.934830,0.919118,0.926907,0.992760
4000,0.022400,0.020119,0.935428,0.919824,0.927560,0.992829
4500,0.018400,0.020033,0.935670,0.921555,0.928559,0.992921
5000,0.022200,0.019722,0.934923,0.923450,0.929151,0.992977


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CORRECT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: INCORRECT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/bert-trained-spellcheck/Bert for spell check - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CORRECT s

TrainOutput(global_step=7820, training_loss=0.023149509611718185, metrics={'train_runtime': 8509.4053, 'train_samples_per_second': 29.379, 'train_steps_per_second': 0.919, 'total_flos': 4.669980666294086e+16, 'train_loss': 0.023149509611718185, 'epoch': 10.0})

In [ ]:
lora_model.save_pretrained("lora_model_adapter")

In [59]:
Data_wiki.loc[1900]

Title              Ramakrishna Mission Sevashrama, Sargachi
Final     Ramakrishna Mission Sevashrama Sargachi is loc...
Errors    Ramakrishna Mission Sevashrama Sargachi were l...
Label     [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
Name: 1900, dtype: object

In [19]:
from peft import PeftConfig, PeftModel

base_model = BertForTokenClassification.from_pretrained("/kaggle/input/bert-trained-spellcheck/Bert for spell check")
inference_model = PeftModel.from_pretrained(copy.deepcopy(base_model), "/kaggle/working/checkpoint-7000/LORA")

In [28]:
base_model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [29]:
inference_model.device

device(type='cpu')

## Lets test the model

In [62]:
import torch
get_incorrect_word(inference_model , "I am in the school now")

'No incorrect words found'

In [63]:
get_incorrect_word(inference_model , "I am in the school yesterday")

['am']

In [86]:
get_incorrect_word(inference_model , "I was in the school yesterday")

'No incorrect words found'

In [65]:
get_incorrect_word(inference_model , "I am in the school tomorrow")

['am']

In [67]:
print(get_incorrect_word(inference_model , "I will be in the school tomorrow"))
print(get_incorrect_word(inference_model , "I shall be in the school tomorrow"))

No incorrect words found
No incorrect words found


In [68]:
text = """On this page you will find our complete 
list of high quality reading comprehension 
worksheets created specially by our team for 
students in grade levels K-12. Our worksheets 
elicit the use of critical thinking skills at 
every level. While some questions ask the reader 
to peruse the passage for particular details, 
most questions involve the use of deductive reasoning, 
conclusion making, logical inference, sequential analysis, 
tonal awareness, and an understanding of scope."""

In [69]:
get_incorrect_word(inference_model , text)

'No incorrect words found'

In [84]:
"""
changes: 

find -> found
students -> studentsss
grade -> graded
inference -> inferenced
sequentiallllyyy -> sequential
tonal -> ton
understanding -> understood

"""


incorrect_text = """On this page you will found our complete 
list of high quality reading comprehension 
worksheets created specially by our team for 
studentsss in graded leveled K-12. Our worksheets 
elicit the use of critical thinking skills at 
every level. While some questions ask the reader 
to peruse the passage for particular details, 
most questions involve the use of deductive reasoning, 
conclusion making, logical inferenced, sequentiallllyyy analysis, 
ton awareness, and an understood of scope."""

In [83]:
get_incorrect_word(inference_model , incorrect_text)

['found',
 'studentsss',
 'graded',
 'leveled',
 'inferenced',
 'sequentiallllyyy',
 'ton',
 'understood']